In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore')
import scalp
from scalp.output import draw
import lmz
import numpy as np
from scipy.sparse import csr_matrix
import scanpy as sc
import seaborn as sns

In [ ]:
# dataset = scalp.data.mkblobs(sizes = [[24,21],[25,20],[23,22]], clusterspread = .3, batchspread = 4)
dataset = scalp.data.mkblobs(sizes = [[40,24,21],[40,25,20],[40,23,22]],  clusterspread = .5, batchspread = 1.5)
#dataset = scalp.data.mkblobs(sizes = [[3,5],[5,3]], clusterspread = .1, batchspread = 4)
for d in dataset:
    d.obsm['umap'] = d.X
stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'], s= 80)

In [ ]:
from scipy.sparse.csgraph import dijkstra
from scalp import graph as sgraph
import pacmap
parm = {'neighbors_total': 30,
         'neighbors_intra_fraction': .3,
         'add_tree': False,
         'epsilon'  : 1e-4, 
          'copy_lsa_neighbors': False, 
         'inter_outlier_threshold': .9,
         'inter_outlier_probabilistic_removal': False}

dataset, graph = scalp.mkgraph(dataset,**parm)
# neggraph = sgraph.negstuff(dataset,**parm)
# graph[graph >= 100] = 2
# graph = csr_matrix(graph).toarray()

#graph[graph==0] = 99
scalp.umapwrap.graph_umap(dataset,graph,label = 'umap', n_components = 2)
# fullgraph = dijkstra(graph)
# scalp.umapwrap.graph_pacmap(dataset,fullgraph,label = 'umap', n_components = 2, neighbors = None, MN  = .2, FP = 2)
# scalp.umapwrap.graph_jax(dataset,graph,label = 'umap', n_components = 2, **parm)
# scalp.umapwrap.graph_umap(dataset,graph,label = 'umap', n_components = 2)

stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'], s= 100)

#sns.heatmap(neggraph.toarray())
#plt.show()

# from sklearn.metrics.pairwise import euclidean_distances
# sns.heatmap(euclidean_distances(stack.obsm['umap']))
# plt.show()

In [ ]:
mask = (graph.toarray()>0 )& (neggraph.toarray() > 0)
st = (csr_matrix(graph),csr_matrix(neggraph))
st[0][mask] = 0
scalp.umapwrap.graph_jax(dataset,st,label = 'umap', n_components = 2)[0].obsm['umap']

In [ ]:
np.linalg.norm([[ 0.      ,    0.        ],
 [-0.00783532 ,-0.0041316 ],
 [-0.07946298 , 0.00789009]], axis = 1)

In [ ]:

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

def plot_sparse_matrix(sparse_matrix, plot_title="Graph"):
    """
    Converts a sparse matrix into a NetworkX graph and plots it using a spring layout.

    Parameters:
    - sparse_matrix (scipy.sparse.csr_matrix): The sparse matrix representing the graph.
    - plot_title (str): Title for the plot.

    Returns:
    - None
    """
    # Convert the sparse matrix to a NetworkX graph
    graph = nx.from_scipy_sparse_array(sparse_matrix)
    # print(graph.edges(data=True))
    # Compute the spring layout
    pos = nx.spring_layout(graph)

    # Plot the graph
    plt.figure(figsize=(8, 8))
    col = stack.obs['label']
    nx.draw(graph, pos, with_labels=True,
            node_size=100, node_color=col,
            font_size=8, font_color='black', font_weight='bold',
            edge_color='gray', linewidths=0.5, alpha=0.7)
    plt.title(plot_title)
    plt.show
    
plot_sparse_matrix(csr_matrix(graph), plot_title="Graph")


In [ ]:

arr = np.arange(9).reshape((3, 3))

np.random.shuffle(arr.T)

arr

In [ ]:
np.sum((graph.toarray()>0 )& (neggraph.toarray() > 0))